In [15]:
from config import *

**Download Dataset**

In [16]:
import shutil
from huggingface_hub import snapshot_download


snapshot_path = snapshot_download(repo_id=DATASET_REPO, repo_type="dataset")
shutil.copytree(snapshot_path, LOCAL_DIR, dirs_exist_ok=True)

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

'./EU_scientific_corpus'

**Process Data**

In [17]:
import xml.etree.ElementTree as ET


def parse_tmx(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    tu_list = []
    for tu in root.iter("tu"):
        segs = [tuv.find("seg").text for tuv in tu.findall("tuv")]
        tu_list.append(tuple(segs))
    return tu_list

In [18]:
import os

data_dir = os.path.join(LOCAL_DIR, DATA_DIR)

for file in os.listdir(data_dir):
    if file.endswith(".tmx"):
        segments = parse_tmx(os.path.join(data_dir, file))
        source_sents = [seg[0] for seg in segments]
        target_sents = [seg[1] for seg in segments]

        os.makedirs(f"{DATA_DIR}/{file[:-3]}")

        with open(f"data/{file[:-3]}/source.txt", "w", encoding="utf-8") as out_file:
            out_file.write("\n".join(source_sents) + "\n")

        with open(f"data/{file[:-3]}/target.txt", "w", encoding="utf-8") as out_file:
            out_file.write("\n".join(target_sents) + "\n")

In [19]:
# Delete original dir
shutil.rmtree(LOCAL_DIR)